In [1]:
import os
import pandas as pd
import yfinance as yf #URL docs: https://pypi.org/project/yfinance/
from datetime import datetime, timedelta
from IPython.display import display, HTML
import yaml   # For sorting and printing dict 

CURR_DIR = os.getcwd() 
print(CURR_DIR)

C:\Users\magnu\dev\aksjeanalyse


In [3]:
# Settings and functions for printing and plotting in notebook
pd.set_option("display.max_rows", None, "display.max_columns", None)

display(HTML("<style>:root {color-scheme: dark;}</style>"))   # Mørke scrollbars for dark mode

def display_df(df):
    # Puts the scrollbar next to the DataFrame
    display(HTML("<div style='height: 250px; overflow: auto; width: fit-content'>" + df.to_html() + "</div>"))

In [119]:
# Collection of functions

def df_to_csv(filename, df):
    df.to_csv(filename, encoding='utf-8', sep=";", index=False)

def add_new_name_symbol_csv_from_oslo_bors(input_csv_file_name):
    df = pd.read_csv(CURR_DIR+"\list_names_of_sectors.csv", sep=';', usecols = ["SectorNames"])
    display_df(df)
    x = input("Exit (q). Velg sektor (0 - "+ str(df.shape[0]-1)+ "). Legg/fjern sektor (n/r):") 
    if x == "n":
        x = input("Skriv inn navn på ny sektor:")
        print("Valgt sektor:", x)
        new_row = pd.DataFrame()
        new_row = pd.DataFrame([x], columns=['SectorNames'])
        df = pd.concat([new_row,df.loc[:]]).reset_index(drop=True)
        df = df.sort_values(by = ["SectorNames"])
        df = df.reset_index(drop=True)
        df.to_csv(CURR_DIR+"\list_names_of_sectors.csv", encoding='utf-8', sep=";", index=False)  
    elif x == "r":
        x = input("Velg sektor (0 - "+ str(df.shape[0]-1)+ "), du vil fjerne:")
        print("Fjerner sektor:", x)
        df = df.drop(int(x))
        df = df.sort_values(by = ["SectorNames"])
        df = df.reset_index(drop=True)
        df.to_csv(CURR_DIR+"\list_names_of_sectors.csv", encoding='utf-8', sep=";", index=False)  
    elif x == "q":
        return None
    elif (0<= int(x) <= df.shape[0]-1):
        name_symbol_df = pd.read_csv(input_csv_file_name, sep=';', usecols = ['Name','Symbol'], skiprows=[1,2,3])
        output_csv_file_name = df["SectorNames"].iloc[int(x)]+".csv"
        name_symbol_df.to_csv(output_csv_file_name, encoding='utf-8', sep=";", index=False)
        print("csv-filen", output_csv_file_name, "er nå opprettet fra csv-filen", input_csv_file_name)
    display_df(df)
    
def get_name_symbol_df_from_csv(print_out=False):
    df = pd.read_csv(CURR_DIR+"\list_names_of_sectors.csv", sep=';', usecols = ["SectorNames"])
    display_df(df)
    x = input(" Velg sektor (0 - "+ str(df.shape[0]-1)+ ").") 
    csv_file_name = df["SectorNames"].iloc[int(x)]+".csv"
    df=pd.read_csv(csv_file_name, sep=';')
    print("Laster all data fra",csv_file_name,"inn i dataframe name_symbol_df")
    if print_out==True:
        print("Utskrift av name_symbol_df. Data fra:",csv_file_name)
        display_df(df)
    return df

def get_tickers_from_selected_symbols(name_symbol_df, selected_indexes, save_history_in_csv_files=False, print_progress_arg=False, period_arg="max", interval_arg="1d"):
    # use "period" instead of start/end. Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max.
    # fetch data by interval (including intraday if period < 60 days). valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    # URL docs: https://pypi.org/project/yfinance/ 
    if selected_indexes == []: # Select symbols from name_symbol_df
        selected_indexes = list(range(0, name_symbol_df.shape[0]))
    selected_symbols_string = "" # String of all selected symbols. 
    for i in selected_indexes:
        selected_symbols_string = selected_symbols_string+" "+name_symbol_df["Symbol"].iloc[i]+".OL"
    tickers = yf.Tickers(selected_symbols_string) # Download all tickers from yfinance
    if save_history_in_csv_files==True:
        TICKER_HISTORY_FOLDER = CURR_DIR+r"\ticker_history_folder"+"\\"   #Name of folder for ticker history data
        for i in selected_indexes:
            symbol = name_symbol_df["Symbol"].iloc[i]+".OL"
            ticker_hist_df = tickers.tickers[symbol].history(period=period_arg, interval = interval_arg)
            ticker_hist_csv_file_name = TICKER_HISTORY_FOLDER + symbol[:-3]+".csv"
            ticker_hist_df.to_csv(ticker_hist_csv_file_name, encoding='utf-8', sep=";", index=False)
            if print_progress_arg == True:
                print("Opprettet:", ticker_hist_csv_file_name, "Navn:",name_symbol_df["Name"].iloc[i]+". Antall rader:", ticker_hist_df.shape[0])
        if print_progress_arg==False:
            print("Ferdig")
    return tickers



In [116]:
# Kode for å lage egne csv-filer for avgrensede sektorer på Oslo børs
# Last ned en sektor-Euronext_Equities_dato.csv-fil ved å bruke filterfunksjonen på Oslo Børs
# URL:  https://live.euronext.com/nb/markets/oslo/equities/list
temp_file_name = CURR_DIR+r"\Euronext_Equities_2023-02-08.csv"            
add_new_name_symbol_csv_from_oslo_bors(temp_file_name)

,SectorNames
0,All_stocks_oslo_bors
1,AlternativeElectricity
2,AlternativeEnergy
3,Banks
4,BasicMaterials
5,ConsumerDiscretionary
6,ConsumerStaples
7,Energy
8,FinancialServices
9,Financials


Exit (q). Velg sektor (0 - 20). Legg/fjern sektor (n/r): 6


csv-filen ConsumerStaples.csv er nå opprettet fra csv-filen C:\Users\magnu\dev\aksjeanalyse\Euronext_Equities_2023-02-08.csv


,SectorNames
0,All_stocks_oslo_bors
1,AlternativeElectricity
2,AlternativeEnergy
3,Banks
4,BasicMaterials
5,ConsumerDiscretionary
6,ConsumerStaples
7,Energy
8,FinancialServices
9,Financials


In [120]:
name_symbol_df = get_name_symbol_df_from_csv(print_out=True)

,SectorNames
0,All_stocks_oslo_bors
1,AlternativeElectricity
2,AlternativeEnergy
3,Banks
4,BasicMaterials
5,ConsumerDiscretionary
6,ConsumerStaples
7,Energy
8,FinancialServices
9,Financials


 Velg sektor (0 - 20). 6


Laster all data fra ConsumerStaples.csv inn i dataframe name_symbol_df
Utskrift av name_symbol_df. Data fra: ConsumerStaples.csv


,Name,Symbol
0,AKER BIOMARINE,AKBM
1,ANDFJORD SALMON,ANDF
2,AQUA BIO TECHNO,ABT
3,ARCTIC BIOSCIENCE,ABS
4,ARCTIC FISH HOLDIN,AFISH
5,ATLANTIC SAPPHIRE,ASA
6,AUSTEVOLL SEAFOOD,AUSS
7,BAKKAFROST,BAKKA
8,BARRAMUNDI GROUP,BARRA
9,BIOFISH HOLDING,BFISH


In [9]:
# Kode for å laste ned tickers fra yahoo (yfinance) og lagre data i egne csv-filer
# Velg hvilke selskaper du vil laste  inn i name_symbol_df ved å fylle ut selected_indexes-listen 
selected_indexes = []  # Select only these indexes in name_symbol_df. For all indexes: set selected_indexes = []
tickers = get_tickers_from_selected_symbols(name_symbol_df, selected_indexes, save_history_in_csv_files=True, print_progress_arg=False, period_arg="max")

Got error from yahoo api for ticker HYNS.OL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- HYNS.OL: No timezone found, symbol may be delisted
Ferdig
